## SCRAPPING DES DONNEES SUR LES LOGEMENTS DE AIRBNB PARIS

le but de cette partie est de scraper les données sur l'ensemble des logements porposés par airbnb Paris du 30 Décembre 2020 au 31 Décembre 2020.

In [ ]:
from bs4 import BeautifulSoup 
from selenium import webdriver
import pandas as pd 
import numpy as np 
import time        
import requests  
import re          
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed 
import itertools
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains 

la première étape consiste à recupérer le code HTLML de la page qui contient l'ensemble des logments à partir de son url. Cette page étant statique, BeautifulSoup sera suffisant.
Par la suite, la classe des differents logements est récupéré. Cette classe est obtenue en inspectant le site web de airbnb Paris qui contient des annonces. 

In [ ]:
def getPage(url):
	result = requests.get(url)
	content = result.content
	return BeautifulSoup(content, features = "lxml")

def getRoomClasses(soupPage):
	rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
	result = []
	for room in rooms:
		result.append(room)
	return result

url_page ="https://www.airbnb.fr/s/Paris--France/homes?adults=1&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&refinement_paths%5B%5D=%2Fhomes&checkin=2020-12-30&checkout=2020-12-31"
page=getPage(url_page)
listing=getRoomClasses(page)

l'objectif suivant est de récupérer un ensemble d'informations sur chaque logment notamment: le nombre de commentaires, l'évaluation, le lien vers le logement, le titre du logement, la description du logement, le nombre de chambre, de lit, de douche ainsi que le prix

In [ ]:
##commentaires
def getNbCommentaire(listing):
	try:
		return listing.find("span", {"class":"_a7a5sx"}).text 
	except:
		return "pas de commentaire"

##evaluation
def getEvaluation(listing):
	try:
		return listing.find("span", {"class":"_10fy1f8"}).text 
	except:
		return "pas d'évaluation"

## lien du logement
def getListingLink(listing):
	try:
		return "http://airbnb.com" + listing.find("a")["href"]
	except:
		return "pas de lien"

## titre du logement
def getListingTitle(listing):
	try:
		return listing.find("meta")["content"]
	except:
		return "pas de titre du logement"

##description du logement
def getTopRow(listing):
	try:
		return listing.find("div", {"class": "_167qordg"}).text
	except:
		return "pas de description du logement"   
    
##information sur le nombre de chambre/douche
def getRoomInfo(listing):
	try:
		return listing.find("div", {"class":"_kqh46o"}).text
	except:
		return "pas d'infos sur chambre/douche"
##prix
def getPrix(listing):
	try:
		return listing.find("div", {"class":"_1fwiw8gv"}).text
	except:
		return "pas d'infos sur le prix"

L’étape suivante consiste à scrapper la liste de tous les logments de Paris. Au lieu d’une seule page, Airbnb présente ses logements sur une quinzaine de pages. Au bas de chaque page, il est possible de cliquer sur un bouton pour passer à la page suivante. Notre objectif est d’automatiser la tâche d’aller à la page suivante. 
Nous commençons par identifier l’objet fléché. l'inspection de la page nous montre qu'il s'agit d'un objet de la classe « _za9j7e » de type « a ». 
Nous automatisons de ce fait, le passage à la page suivante et l'extraction de l'URL de chaque page

In [ ]:
def findNextPage(soupPage):
	try:
		nextpage = "https://airbnb.com" + soupPage.find("a", {"class": "_za9j7e"})["href"]
	except:
		nextpage = "no next page"
	return nextpage

def getPages(url):
	result = []
	while url != "no next page": 
		page = getPage(url)
		result = result + [page]
		url = findNextPage(page)
	return result


le code suivant permet de créer une base de données comportant toutes les informations que l'on desire scrapper sur l'ensemble des logements de Airbnp paris

In [ ]:
## extraction des infos de tous les logements d'une page
def extractinfo(page):
    df = pd.DataFrame(columns = ['title', 'link',"nbComments","prix","topRow","evaluation","roomInfo"]) ##,"evaluation", ,'nbComments'
    new=[]
    listing=getRoomClasses(page)
    for i in range(1, len(listing)):
        new.append(getListingTitle(listing[i]))
        new.append(getListingLink(listing[i]))
        new.append(getNbCommentaire(listing[i]))
        new.append(getPrix(listing[i]))
        new.append(getTopRow(listing[i]))
        new.append(getEvaluation(listing[i]))
        new.append(getRoomInfo(listing[i]))
        df.loc[i]=new
        new=[]
    return df

### Scrapping proprement dit des informations sur l'ensemble des logements présents dans l'annonce de Airbnb Paris

In [ ]:
## Scrapper toutes les infos sur les logements
def extractPages(url):
	pages = getPages(url)
	df =extractinfo(pages[0])
	for pagenumber in range(1, len(pages)):
		df = df.append(extractinfo(pages[pagenumber]))
	return df

In [ ]:
##base de tous les logements avec les évaluations
data=pd.DataFrame.from_dict(extractPages(url_page))
data.head(5)

Pour éviter de lancer l'ensemble des codes à chaque fois, nous décidons d'exporter la première partie de notre jeu de données

In [ ]:
len(data.index)
import csv
data.to_csv('D:\Python\dataListing.csv', index = True)

## scrapping des commentaires de chaque logement

Pour scrapper les commentaires de chaque logement, il incombe d'ouvrir individuellement chacun d'eux. Les liens vers la page détaillée de chaque logement obtunus lors du premier crapping sont utilisés. 
Malheureusement, notre méthode précédente de scrapping d’un site web statique ne peut pas être utilisée car Airbnb charge le contenu de ces pages détaillées sur la page Web à l’aide de javascript. Nous utilisons donc dans la suite la package *Selenium*

le pilote Web utilisé est Chrome. l'étape suivante est donc de configurer ce pilote. On définit également un temps d'attente necessaire au chargement de chaque page. Nous avons défini un temps d'attente de 5 secondes

In [ ]:
## installer chrome driver
driver_path = 'D:/chromedriver.exe'
opt = webdriver.ChromeOptions()
opt.add_experimental_option('w3c', False)
driver = webdriver.Chrome(executable_path=driver_path,options=opt)

##Initialiser le driver de Selenium
def setupDriver(url, waiting_time = 5):
	driver = webdriver.Chrome(options=opt)
	driver.get(url)
	time.sleep(waiting_time) 
	return driver

La page de chaque logement permet d'avoir les commentaires en cliquant sur un lien. ce lien nous dirige vers une autre page dans laquelle l'extraction effective a lieu.
Le code suivant permet de charger chaque page detaillée et recupérer le lien de la page des commentaires

In [ ]:
## charger la page de chaque logement
def getJSpage(url):
	driver = setupDriver(url)
	html = driver.page_source
	driver.close()
	return BeautifulSoup(html, features="lxml")

# lien de la page des commentaires de chaque listing
def getCommentLink(soupPage):
	try:
		return "https://www.airbnb.fr" + soupPage.find("div", {"class": "_19qg1ru"}).find("a")["href"]
	except:
		return "pas de lien"

Une fois la page des commentaires ouverte, il incombe de déterminer la classe qui sera utilisée pour l'extraction. l'inspection de la page revèle qu'il s'agit d'un objet de classe "_1gjypya". Dans cette objet, sont présents les commentaires des clients ainsi que ceux de Airbnb. Nous allons nous restreindre aux seuls commentaires des clients en utilisant la classe "_1y6fhhr"

In [ ]:
##Recuperer l'ensemble des commentaires
def getRoomComments(soupage):
    #try
    crooms=soupage.findAll('div', {'class': '_1gjypya'})
    result=[]
    for room in crooms:
        result.append(room)
    return result

def getcomments(result):
    comments=[]
    for i in result:
        try:
            comments.append(i.find('div', {'class': '_1y6fhhr'}).text)
        except: 
            pass
    return comments

Précisons que l'ensemble des commentaires d'un logement s'ouvre dans une boite de dialogue qu'il faudra au préalable faire défiler ou scroller complètement avant l'extraction. le code suivant permet donc de scroller la boite de dialogue des commentaires et recupèrer le code source de la page.  

In [ ]:
##Scrolling
def ScrollPagecomments(url):
    driver=setupDriver(url)
    num_current_comment_found = 0
    pre_scroll_num_of_comment=-1
    comment_class_when_scrolling  = '_1gjypya'
    time.sleep(5)
    while (num_current_comment_found != pre_scroll_num_of_comment) :           
                visible_comments = driver.find_elements_by_class_name(comment_class_when_scrolling)
                pre_scroll_num_of_comment= len(visible_comments)
                try:
                    last_visible_comment = visible_comments[-1]
                    actions = ActionChains(driver)
                    actions.move_to_element(last_visible_comment)
                    actions.perform()
                    time.sleep(2)
                    last_visible_comment.location_once_scrolled_into_view
                    visible_comments = driver.find_elements_by_class_name(comment_class_when_scrolling)
                    num_current_comment_found = len(visible_comments)
                except:
                    pass
    html=driver.page_source
    driver.close()
    return BeautifulSoup(html,features="lxml")

Le code suivant met ensemble les fonctions intermediaires précedemment présentées afin de constituer une base de données qui contient pour chaque logement l'ensemble des commentaires. Elle prend en entrée la base de données obtenue à l'issue du premier scrapping. La base est sauvegardée au fur et à mesure que l'extraction s'effectue. 

In [ ]:
def ExtractionCommentaire(data):
    ind=[]
    com=[]
    DataComments=[]
    i=0
    for link in data['link']:
        soupPage=getJSpage(link)
        url_comment=getCommentLink(soupPage)
        if (url_comment!="pas de lien"):
            js=ScrollPagecomments(url_comment)
            DataComments=getcomments(getRoomComments(js))
            #a=getcomments(getRoomComments(js))
        else :
            DataComments=["pas de commentaire"]
        i=i+1
        ind=ind+list(itertools.repeat(i, len(DataComments)))
        com=com + DataComments
        database=pd.DataFrame.from_dict({'index':ind, 'commentaires':com})
        database.to_csv('D:\Python\database.csv', index = False, encoding = 'utf-8')
    return database

Cette base de commentaires est utilisée dans la suite pour faire du NLP (Natural Language Processing)

In [ ]:
ExtractionCommentaire(data)